In [2]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

### Competencias

### Tipo Personagem

### Assunto

### Classe

### Classe assunto

### Tipo movimento

### Tipo Andamento

### Cargo

### Funcionário

### Tipo Decisão Recurso

### Tipo Ato Juiz

### Ato Juiz

### Tipo Documento

### Processo

#### Obtem as serventias com função de Juizado Especial Cível das comarcas de Bangu e Campo Grande

In [ ]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec

#### Obtem a lista de planilhas para processar resultantes codigos de serventias e do intervalo de anos selecinados 

#### Processa planilhas do processo a partir dos servs_anos selecionados e realiza post requests concorrentes

####  carrega os processos que por algum motivo não tenham sido processados anteriormente

##### Processa Codigo do CNJ do processo

In [ ]:
usecols=['ID_PROC','COD_CNJ','COD_PROC']
cod_procs = list(map(lambda x: x.cod_proc, tj.Processo.objects.filter(cod_cnj=None)))
processos_unicos = pd.DataFrame(columns=usecols)
for df in TJData.read_csv(TJData.csv['processo_unico'], usecols=usecols, chunksize=50000):
    df = df[df.COD_PROC.isin(cod_procs)]
    processos_unicos = pd.concat([processos_unicos,df])
processos_unicos = processos_unicos.rename(columns=lambda x : x.lower())
processos_unicos.cod_cnj = TJData.to_utf8_bytes(processos_unicos.cod_cnj)
processos_unicos.cod_proc = TJData.to_utf8_bytes(processos_unicos.cod_proc)
processos_unicos = TJData.to_json(processos_unicos)
r_patch = api.concurrent_request(api.patch,'processos',processos_unicos, detail = 'id_proc')

In [4]:
tj.Processo.objects.filter(cod_cnj=None)

153009

In [20]:
r_patch

[({'id_proc': 108330129,
   'cod_cnj': '0001965-24.2013.8.19.0205',
   'cod_proc': '2013.205.001567-5'},
  <Response [200]>),
 ({'id_proc': 108331497,
   'cod_cnj': '0001462-06.2013.8.19.0204',
   'cod_proc': '2013.204.001456-0'},
  <Response [200]>),
 ({'id_proc': 108332240,
   'cod_cnj': '0001470-80.2013.8.19.0204',
   'cod_proc': '2013.204.001464-9'},
  <Response [200]>),
 ({'id_proc': 108332422,
   'cod_cnj': '0001476-87.2013.8.19.0204',
   'cod_proc': '2013.204.001470-4'},
  <Response [200]>),
 ({'id_proc': 108331800,
   'cod_cnj': '0001465-58.2013.8.19.0204',
   'cod_proc': '2013.204.001459-5'},
  <Response [200]>),
 ({'id_proc': 108290536,
   'cod_cnj': '0000722-48.2013.8.19.0204',
   'cod_proc': '2013.204.000720-7'},
  <Response [200]>),
 ({'id_proc': 108332321,
   'cod_cnj': '0001472-50.2013.8.19.0204',
   'cod_proc': '2013.204.001466-2'},
  <Response [200]>),
 ({'id_proc': 108332126,
   'cod_cnj': '0001467-28.2013.8.19.0204',
   'cod_proc': '2013.204.001461-3'},
  <Response [

In [18]:
r.request.url

'http://0.0.0.0:8000/api/models/processos/id_proc/'